In [1]:
!pip install extr-ds
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install seqeval


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/507.1 kB ? eta -:--:--
   ------- -------------------------------- 92.2/507.1 kB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 337.9/507.1 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 507.1/507.1 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/115.3 kB ? eta -:--:--
   ---------------------------------------- 115.3/115.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/364.6 kB ? eta -:--:--
   --------------------------------------- 364.6/364.6 kB 23.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   - -------------------------------------- 1.1/24.8 MB 22.7 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/24.8 MB 34.8 MB/s eta 0:00:01
   ----- ---------------------------------- 3.7/24.8 MB 38.8 MB/s eta 0:00:01
   ----- ---------------------------------- 3.7/24.8 MB 23.3 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached seqeval-1.2.2-py3-none-any.whl



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import GPUtil
GPUtil.getAvailable()

[0]

In [1]:
import torch
use_cuda = torch.cuda.is_available()

In [2]:
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

In [14]:
import json

import random
from datasets import Dataset
from transformers import DataCollatorForTokenClassification
from extr_ds.manager.utils.filesystem import load_document
import numpy
import evaluate
from transformers.keras_callbacks import KerasMetricCallback
import tensorflow as tf
from transformers import AutoTokenizer, \
    TFAutoModelForTokenClassification
from transformers import pipeline
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
model_checkpoint = 'bert-base-cased'
model_output_checkpoint = 'transformers/nfl_pbp_token_classifier'

In [3]:
epochs = 1

labels = [
    "yes",
    "no"
]

label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [11]:
def align_labels(tokenized_inputs, label_list):
    labels = []
    for word_idx in tokenized_inputs.word_ids(batch_index=0):
        label_id = -100
        if word_idx is not None:
            label = label_list[word_idx]

            label_id = label2id[label]

        labels.append(label_id)
        # previous_word_idx = word_idx

    return labels


def get_dataset(tokenizer, model):
    def tokenize_and_align_labels(record):
        tokenized_inputs = tokenizer(
            record['tokens'],
            truncation=True,
            is_split_into_words=True
        )

        tokenized_inputs['labels'] = align_labels(
            tokenized_inputs,
            record['labels']
        )

        return tokenized_inputs

    ents_dataset = json.loads(
        load_document('dataset/train_labels.json')
    )

    # random.shuffle(ents_dataset)

    pivot = int(len(ents_dataset) * .95)
    data_collator = DataCollatorForTokenClassification(
        tokenizer,
        return_tensors='tf'
    )

    train_dataset = Dataset.from_list(ents_dataset[:pivot])
    tf_train_set = model.prepare_tf_dataset(
        train_dataset.map(
            tokenize_and_align_labels,
            batched=False
        ),
        shuffle=True,
        collate_fn=data_collator,
    )

    test_dataset = Dataset.from_list(ents_dataset[pivot:])
    tf_test_set = model.prepare_tf_dataset(
        test_dataset.map(
            tokenize_and_align_labels,
            batched=False
        ),
        shuffle=True,
        collate_fn=data_collator,
    )

    return tf_train_set, tf_test_set


seqeval = evaluate.load('seqeval')


def compute_metrics(preds):
    predictions, actuals = preds
    predictions = numpy.argmax(predictions, axis=2)

    results = seqeval.compute(
        predictions=[
            [labels[p] for p, l in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, actuals)
        ],
        references=[
            [labels[l] for p, l in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, actuals)
        ]
    )

    return {
        key: results[f'overall_{key}']
        for key in ['f1', 'accuracy']
    }

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint
)

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

tf_train_set, tf_test_set = get_dataset(tokenizer, model)

callbacks = [
    KerasMetricCallback(
        metric_fn=compute_metrics,
        eval_dataset=tf_test_set
    ),
    tf.keras.callbacks.EarlyStopping(monitor='f1', patience=3)
]

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4204 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [13]:
model.fit(
    x=tf_train_set,
    validation_data=tf_test_set,
    epochs=epochs,
    callbacks=callbacks
)

525/525 [==============================] - 781s 1s/step - loss: 0.3117 - val_loss: 0.2140 - f1: 0.6428 - accuracy: 0.9075


In [15]:
for model_to_save in [tokenizer, model]:
  model_to_save.save_pretrained(model_output_checkpoint)

In [16]:
classifier = pipeline(
    'ner',
    model=model_output_checkpoint,
    aggregation_strategy='simple'
)

with open('dataset/test_labels.json') as f:
    test_data = json.load(f)


responses = classifier(test_data)

Some layers from the model checkpoint at transformers/nfl_pbp_token_classifier were not used when initializing TFBertForTokenClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at transformers/nfl_pbp_token_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [17]:
with open("output/test_modelip.txt", "w") as outfile:
    for response in responses:
        for res in response:
            if res["entity_group"] == 'yes':
                for sentiment in str(res['word']).split(" "):
                    outfile.write('1.0\n')
            else:
                for sentiment in str(res['word']).split(" "):
                    outfile.write('0.0\n')

In [18]:
with open('output/test_modelip.txt') as f:
    output = f.readlines()

with open('output/answer.txt', 'w') as f:
    for i in range(1, 1581):
        f.write('neutral\n')
    for i in range(1581, 9271):
        f.write('0.0\n')
    for line in output:
        f.write(line)
